In [ ]:
# Import libraries necessary for this project
import os, sys, calendar
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import preprocessing
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression
from sklearn.pipeline import FeatureUnion

from xgboost.sklearn import XGBRegressor  
import scipy.stats as st

from IPython.display import display
import matplotlib.pyplot as plt
import datetime as dt
from time import time
from collections import OrderedDict
from fbprophet import Prophet

types = {'id': 'int32', 'item_nbr': 'int32', 'store_nbr': 'int8', 'onpromotion': bool}

%matplotlib inline

# Data Preprocess
Fuction data_preprocess() includes:
1. import training data, and fill unit_sales of missing items of stores at some dates with 0
2. import (public) testing data. Kaggle saves targets of this part for model evaluation
3. Combine training data and public testing data
4. Feature engineering
    - Predict transactions at dates of testing data
    - Compute moving average
    - numerical encoding categorical variables
5. Split data back to trainign data and public testing data

In [ ]:
def data_preprocess(store):
    ### Load training data and testing data, and put them together for encoding ###
    filename_train = 'train_store_' + str(store) + '.csv'
    train_data_raw = pd.read_csv('input_each_store_start20170601/' + filename_train, usecols = [1,2,3,4,5],
                                 parse_dates=['date'], dtype=types, #skiprows=range(1,200000),
                                 infer_datetime_format = True,
                                 converters={'unit_sales':lambda u: float(u) if float(u)>0 else 0},)
    print("Raw training data has {} samples with {} features each.".format(*train_data_raw.shape))

    u_dates = train_data_raw.date.unique(); u_stores = train_data_raw.store_nbr.unique(); u_items = train_data_raw.item_nbr.unique()
    train_data = train_data_raw.set_index(['date', 'store_nbr', 'item_nbr'])
    train_data = train_data.reindex(pd.MultiIndex.from_product([u_dates, u_stores, u_items], names=['date', 'store_nbr', 'item_nbr']))
    del u_dates, u_stores, u_items
    train_data.unit_sales.fillna(0, inplace=True)
    train_data.onpromotion.fillna(0, inplace=True)
    train_data['unit_sales'] = train_data['unit_sales'].apply(np.log1p)
    train_data.reset_index(inplace=True)
    
    filename_test = 'test_store_' + str(store) + '.csv'
    test_data_raw = pd.read_csv('input_each_store_start20170701/' + filename_test, usecols=[1,2,3,4], parse_dates=['date'], dtype=types)
    print("Public testing data has {} samples with {} features each.".format(*test_data_raw.shape))
    test_data_raw.dropna(inplace=True)

    data_all = pd.concat([train_data, test_data_raw])
    data_all.drop('store_nbr', axis=1, inplace=True)

    ### Extend training data information ###
    items = pd.read_csv("input/items.csv", dtype={'perishable': np.dtype('int8')})
    items.dropna(inplace=True)
    data_all= pd.merge(data_all,items, right_on='item_nbr',left_on='item_nbr',how='left')
    
    transactions = pd.read_csv("input/transactions.csv", parse_dates=['date'])
    transcations_store = transactions[transactions['store_nbr']==store].dropna()
    transaction_pred = predict_transactions(store)
    transactins_all = pd.concat([transcations_store.drop('store_nbr', axis=1), transaction_pred])    
    data_all = pd.merge(data_all, transactins_all, left_on=['date'], right_on=['date'], how='left')
    
    data_all_to_group = data_all.set_index('date')
    data_all_grouped = data_all_to_group.groupby(['item_nbr'])
    for i in [1,2,4,7,14,28,56]:
        window = str(i) + 'd'
        col = 'MA' + str(i)
        temp = data_all_grouped['unit_sales'].rolling(window=window).mean().fillna(0).to_frame(col).reset_index()
        data_all = pd.merge(data_all, temp, left_on=['date', 'item_nbr'], right_on = ['date', 'item_nbr'], how = 'left')
    data_all['MA'] = data_all[[f for f in list(data_all) if "MA" in f]].mean(axis=1).fillna(0)    
    data_all.drop(['MA{}'.format(i) for i in [1,2, 4,7,14,28,56]], axis = 1, inplace = True)
    
    ### Encoding ###
#     #One_hot encoding
#     feature_one_hot_coded = pd.get_dummies(data_all, columns = ['item_nbr', 'family', 'class'])
    #Binary encoding
    categorical_items = ['family', 'class']
    for col in categorical_items:
#         binaryEncode = preprocessing.LabelBinarizer().fit_transform(data_all[col]).astype(np.int8)
#         df_temp = pd.DataFrame(binaryEncode)
#         df_temp = df_temp.add_prefix(col)
#         data_all = pd.concat([data_all, df_temp], axis=1)
#         del data_all[col]
        data_all[col] = pd.Categorical(data_all[col]).codes
    data_all['onpromotion'] = preprocessing.Binarizer().fit_transform(data_all['onpromotion'].values.reshape(-1,1)).astype(np.int8)
    data_all['perishable'] = data_all['perishable'].astype(np.int8)
    data_all['transactions'] = preprocessing.MinMaxScaler().fit_transform(data_all['transactions'].values.reshape(-1,1))
    
    
    ### Separate training out to train PCA ####
    train_data = data_all[:train_data.shape[0]].copy()
    train_target = train_data['unit_sales'].values

    
    train_processed = data_all[:train_data.shape[0]].copy()
#     train_processed['unit_sales'] = train_target
    test_processed = data_all.drop('unit_sales', axis=1)[train_data.shape[0]:].copy()
    
    return [train_processed, test_processed]

# Transction Prediction
Holidays and events are factors affecting transactions. Differnt holidays and events have different affecting scope, such as national, regional, and local. This means some holidays and events only have effects on some specific stores. In this part, I firstly locate a store, and then filter out geographically related holidays and events. In holiday, payoff days and earthquake on April 16, 2016 are also included.

In [ ]:
def store_loc_search(store):
    #Locate a store
    location = {}    
    stores_raw = pd.read_csv('input/stores.csv')
    location['city'] = stores_raw.loc[stores_raw['store_nbr']==store, 'city'].values[0]
    location['state'] = stores_raw.loc[stores_raw['store_nbr']==store, 'state'].values[0]
    return location

def holidays(store):
    #holidays_events happens at the region of a store
    city,state = store_loc_search(store)
    holidays_events_raw = pd.read_csv('input/holidays_events.csv', parse_dates=['date'])
    mask = (holidays_events_raw['date'] >= '2015-01-01') & (holidays_events_raw['transferred'] == False)&(
        ((holidays_events_raw['locale']=='Local')&(holidays_events_raw['locale_name']==city))|
        ((holidays_events_raw['locale']=='Reginal')&(holidays_events_raw['locale_name']==state))|
        (holidays_events_raw['locale']=='National')
    )
    holidays_events = pd.DataFrame({
        'holiday':holidays_events_raw[mask]['type'],
        'ds':holidays_events_raw[mask]['date'],
        'lower_window':0,
        'upper_window':1,
    })
    
    #payoff days: Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month.
    payoff_dates = []
    for year in range(2015, 2018):
        for month in range(1,13):
            payoff_dates.append(str(year)+'-'+str(month)+'-15')
            payoff_dates.append(str(year)+'-'+str(month)+'-'+str(calendar.monthrange(year,month)[1]))
    payoffs = pd.DataFrame({
      'holiday': 'playoff',
      'ds': pd.to_datetime(payoff_dates),
      'lower_window': 0,
      'upper_window': 1,
    })
    #earthquak at 2016-04-16
    earthquake = pd.DataFrame({
      'holiday': 'earthquake',
      'ds': pd.to_datetime(['2016-04-16']),
      'lower_window': 0,
      'upper_window': 14,
    })    
    #Put holiday_events and payoff_dates together
    holidays = pd.concat((holidays_events, payoffs, earthquake)).fillna(0).sort_values(['ds'])
    return holidays

def transactions(store):
    #History transaction data of this store
    transactions_raw = pd.read_csv('input/transactions.csv', parse_dates = ['date'])
    transactions = transactions_raw.loc[transactions_raw['store_nbr']==store].copy()
    transactions.drop(['store_nbr'], axis=1, inplace=True)
    transactions = transactions.rename(columns={'date': 'ds','transactions': 'y'})
    return transactions

def predict_transactions(store):
    #Use FBProphet to predict transctions of testing dates
    phophet_model = Prophet(holidays=holidays(store))
    transactions_store = transactions(store)
    phophet_model.fit(transactions_store)
    future = phophet_model.make_future_dataframe(periods=16, freq='d')
    forecast = phophet_model.predict(future)
    transaction_pred = forecast.loc[forecast['ds']>='2017-08-16',['ds', 'yhat']]
    transaction_pred = transaction_pred.rename(columns={'ds': 'date', 'yhat': 'transactions'})
    return transaction_pred

# Metric


In [ ]:
def nwrmsle(ground_truth, predictions, w):
    return mean_squared_error(ground_truth, predictions, sample_weight=w)**0.5

nwrmsle_scorer = make_scorer(score_func=mean_squared_error, greater_is_better=False)

# Model Training
In this part, training data are split into two parts, one part is used to train models, the other part is used to validate the trained models.  
Here extreme gradient boosted regression model used, and ramdomized search cross validation is employed to optimize the model.

In [ ]:
def train_predict(xgbreg, data_processed):
    result = {}
    train_data = data_processed[data_processed['date']<'2017-08-01'].copy()
    train = train_data
    train['unit_sales(t-1)'] = train.groupby('item_nbr')['unit_sales'].shift(1)
    train['unit_sales(t-3)'] = train.groupby('item_nbr')['unit_sales'].shift(3)
    train['unit_sales(t-7)'] = train.groupby('item_nbr')['unit_sales'].shift(7)  

#     valid = data[data['date']>='2017-08-01']
#     y_valid = valid['unit_sales']; X_valid = valid.drop(['date','unit_sales'], axis=1)
    y_train = train['unit_sales']; X_train = train.drop(['date','unit_sales'], axis=1)
    w_train = X_train['perishable'].apply(lambda x: 1.25 if x else 1).values
    

    one_to_left = st.beta(10, 1)  
    from_zero_positive = st.expon(0, 50)

    params = {  
        "n_estimators": st.randint(40, 80),
        "max_depth": st.randint(40, 80),
        "learning_rate": st.uniform(0.1, 0.4),
        "colsample_bytree": one_to_left,
        "subsample": one_to_left,
        "gamma": st.uniform(0, 10),
        'reg_alpha': from_zero_positive,
        "min_child_weight": from_zero_positive,
    }
    optimized_XGB = RandomizedSearchCV(xgbreg, params, scoring = 'neg_mean_squared_error', 
                            n_jobs=8, pre_dispatch='2*n_jobs', cv=5, n_iter=50, return_train_score=True)  
    optimized_XGB.fit(X_train, y_train, sample_weight=w_train)
    
    target_true = data_processed[data_processed['date']>='2017-08-01']['unit_sales']
    w_test = data_processed[data_processed['date']>='2017-08-01']['perishable'].apply(lambda x: 1.25 if x else 1).values
    
    base = dt.datetime.strptime('2017-08-01', "%Y-%m-%d")
    date_list = [base + dt.timedelta(days=x) for x in range(0, 16)]
    for date in date_list:
        past_data = data_processed[data_processed['date']<=date].copy()
        past_data['unit_sales(t-1)'] = past_data.groupby('item_nbr')['unit_sales'].shift(1)
        past_data['unit_sales(t-3)'] = past_data.groupby('item_nbr')['unit_sales'].shift(3)
        past_data['unit_sales(t-7)'] = past_data.groupby('item_nbr')['unit_sales'].shift(7)
        
        X_valid = past_data[past_data['date']==date].drop(['date','unit_sales'], axis=1)
        X_valid['unit_sales'] = optimized_XGB.predict(X_valid)
        data_processed.loc[data_processed['date']==date,'unit_sales'] = X_valid['unit_sales'] 
    
    target_pred = data_processed[data_processed['date']>='2017-08-01']['unit_sales']
    score_valid = mean_squared_error(target_true, target_pred, sample_weight=w_test)
    
    result['learner'] = optimized_XGB
    result['prediction'] = data_processed[data_processed['date']>='20170801'][['date','item_nbr','unit_sales']]
    result['prediction'].to_csv('pred_20171208.csv')

    print(np.sqrt(np.abs(optimized_XGB.best_score_)), np.sqrt(score_valid))

    return result

# Model Training for Kaggle
To fully take advantages of training data, all training data are used in this function

In [ ]:
def train_predict_kaggle(xgbreg, data_processed):
    result = {}
    train_data = data_processed[data_processed['date']<'2017-08-16'].copy()
    train = train_data
    train['unit_sales(t-1)'] = train.groupby('item_nbr')['unit_sales'].shift(1)
    train['unit_sales(t-3)'] = train.groupby('item_nbr')['unit_sales'].shift(3)
    train['unit_sales(t-7)'] = train.groupby('item_nbr')['unit_sales'].shift(7)  

    y_train = train['unit_sales']; X_train = train.drop(['date','unit_sales'], axis=1)
    w_train = X_train['perishable'].apply(lambda x: 1.25 if x else 1).values
    

    one_to_left = st.beta(10, 1)  
    from_zero_positive = st.expon(0, 50)

    params = {  
        "n_estimators": st.randint(10, 40),
        "max_depth": st.randint(10, 40),
        "learning_rate": st.uniform(0.05, 0.4),
        "colsample_bytree": one_to_left,
        "subsample": one_to_left,
        "gamma": st.uniform(0, 10),
        'reg_alpha': from_zero_positive,
        "min_child_weight": from_zero_positive,
    }
    optimized_XGB = RandomizedSearchCV(xgbreg, params, scoring = 'neg_mean_squared_error', 
                            n_jobs=8, pre_dispatch='2*n_jobs', cv=5, n_iter=50, return_train_score=True)  
    optimized_XGB.fit(X_train, y_train, sample_weight=w_train)
        
    base = dt.datetime.strptime('2017-08-01', "%Y-%m-%d")
    date_list = [base + dt.timedelta(days=x) for x in range(0, 16)]
    for date in date_list:
        past_data = data_processed[data_processed['date']<=date].copy()
        past_data['unit_sales(t-1)'] = past_data.groupby('item_nbr')['unit_sales'].shift(1)
        past_data['unit_sales(t-3)'] = past_data.groupby('item_nbr')['unit_sales'].shift(3)
        past_data['unit_sales(t-7)'] = past_data.groupby('item_nbr')['unit_sales'].shift(7)
        
        X_valid = past_data[past_data['date']==date].drop(['date','unit_sales'], axis=1)
        X_valid['unit_sales'] = optimized_XGB.predict(X_valid)
        data_processed.loc[data_processed['date']==date,'unit_sales'] = X_valid['unit_sales'] 
    
    result['learner'] = optimized_XGB
    result['prediction'] = data_processed[data_processed['date']>='2017-08-16'][['date','item_nbr','unit_sales']]

    return result

# XGB -- Main

In [ ]:
results = {}
store_list = range(1,2)
predictions = pd.DataFrame(0, index = range(125497040, 128867504), columns = ['unit_sales'])
predictions.index.name = 'id'

regressor = XGBRegressor(objective='reg:linear', nthreads=-1)
reg_name = regressor.__class__.__name__

file_submission_temp = reg_name + dt.datetime.today().strftime("%Y-%m-%d") + '_temp.csv'
file_submission = reg_name + dt.datetime.today().strftime("%Y-%m-%d") + '.csv'
kaggle = False

for store in store_list:
    print("------------------------------")
    start = time()
    print("Start Store {}:".format(store))
    train_processed, test_processed = data_preprocess(store)
    print("Processed training set has {} samples with {} variables.".format(*train_processed.shape))
    
    if not kaggle:
        results[store]= train_predict(regressor, train_processed)
    else:
        results[store]= train_predict_kaggle(regressor, train_processed)
        filename_test = 'test_store_' + str(store) + '.csv'
        test_data_raw = pd.read_csv('input_each_store_start20170701/' + filename_test, parse_dates=['date'], dtype=types)
        test_data = pd.merge(test_data_raw, results[store]['prediction'], left_on = ['date', 'item_nbr'], right_on = ['date', 'item_nbr'], how='left')
        test_data['unit_sales']= [np.expm1(p) for p in test_data['unit_sales']]
        prediction_store = test_data[['id', 'unit_sales']].set_index(['id'])

        predictions.loc[prediction_store.index] = prediction_store
        if not os.path.isfile(file_submission_temp):
            prediction_store.to_csv(file_submission_temp, float_format='%.3f')
        else:
            prediction_store.to_csv(file_submission_temp, mode = 'a', header = False, float_format='%.3f')
    print("Store {} is complete in {:.2f} min.".format(store, (time()-start)/60))

if kaggle:
    predictions.to_csv(file_submission, float_format='%.3f')
    filename_result = reg_name+'_result.sav'
    pickle.dump(results, open(filename_result, 'wb'))

In [ ]:
### Plot errorboar of mean_test_score
# x = range(len(results[1]['learner'].cv_results_['mean_test_score']))
# y = np.sqrt(np.abs(results[1]['learner'].cv_results_['mean_test_score']))
# e = np.sqrt(results[1]['learner'].cv_results_['std_test_score'])
# plt.errorbar(x, y, e, linestyle='None', marker='^')
# plt.xlabel('iteration')
# plt.ylabel('score')
# plt.title('Errorbar plot of mean test score')